In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

**Read in the Image and Make a Copy**

In [2]:
img = cv2.imread('../meta/road_image.jpg')
img_copy = np.copy(img)

**Create an empty space for the results to be drawn**

In [3]:
marker_image = np.zeros(img.shape[:2],dtype=np.int32)
segments = np.zeros(img.shape,dtype=np.uint8)

**Create colors for Markers**

In [4]:
from matplotlib import cm

def create_bgr(i):
    r, g, b = np.array(cm.tab10(i))[:3]*255
    return (b, g, r)

In [5]:
colors = []
for i in range(10):
    colors.append(create_bgr(i))

**Setting Up Callback Function**

In [6]:
# Numbers 0-9
n_markers = 10

# Default settings
current_marker = 1
marks_updated = False

In [7]:
def mouse_callback(event, x, y, flags, param):
    global marks_updated 

    if event == cv2.EVENT_LBUTTONDOWN:
        
        # TRACKING FOR MARKERS
        cv2.circle(marker_image, (x, y), 10, (current_marker), -1)
        
        # DISPLAY ON USER IMAGE
        cv2.circle(img_copy, (x, y), 10, colors[current_marker], -1)
        marks_updated = True

In [8]:
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', mouse_callback)

while True:
    
    # SHow the 2 windows
    cv2.imshow('WaterShed Segments', segments)
    cv2.imshow('Image', img_copy)
        
        
    # Close everything if Esc is pressed
    k = cv2.waitKey(1)

    if k == 27:
        break
        
    # Clear all colors and start over if 'c' is pressed
    elif k == ord('c'):
        img_copy = img.copy()
        marker_image = np.zeros(img.shape[0:2], dtype=np.int32)
        segments = np.zeros(img.shape,dtype=np.uint8)
        
    # If a number 0-9 is chosen index the color
    elif k > 0 and chr(k).isdigit():
        # chr converts to printable digit
        n = int(chr(k))
        if 1 <= n <= n_markers:
            current_marker = n
    
    # If we clicked somewhere, call the watershed algorithm on our chosen markers
    if marks_updated:
        
        marker_image_copy = marker_image.copy()
        cv2.watershed(img, marker_image_copy)
        
        segments = np.zeros(img.shape,dtype=np.uint8)
        
        for color_ind in range(n_markers):
            segments[marker_image_copy == (color_ind)] = colors[color_ind]
        
        marks_updated = False
        
cv2.destroyAllWindows()